In [21]:
using CSV
using DataFrames
using Dates
using Statistics
using LinearAlgebra
using JuMP
using Gurobi

data_dir = "./data"

# ─── HELPERS ───────────────────────────────────────────────────────────────────

function compute_returns(prices::Vector{Float64})
    diff(prices) ./ prices[1:end-1]
end

# ─── LOAD, COMPUTE RETURNS & LIQUIDITY ────────────────────────────────────────

csv_files    = filter(f->endswith(lowercase(f), ".csv"), readdir(data_dir; join=true))
n            = length(csv_files)
dfs_returns  = Vector{DataFrame}(undef, n)
liquidity    = Vector{Float64}(undef, n)
names        = Vector{String}(undef, n)

for (i, file) in enumerate(csv_files)
    # extract company name from filename, e.g. "boeing.csv" → "boeing"
    names[i] = split(basename(file), ".")[1]

    df = CSV.read(file, DataFrame)
    df.Date = Date.(df.Date, dateformat"u d, Y")
    sort!(df, :Date)

    rs = compute_returns(df.Adj_Close)
    vs = df.Volume[2:end]

    sub = df[2:end, [:Date]]
    sub.:return = rs

    dfs_returns[i] = sub
    liquidity[i]   = mean(vs)
end

# rename return columns r1…rn
for i in 1:n
    rename!(dfs_returns[i], :return => Symbol("r$i"))
end

# join on Date
rets = reduce((a,b)->innerjoin(a,b,on=:Date), dfs_returns)

# ─── COMPUTE μ AND Σ ──────────────────────────────────────────────────────────

μ     = [ mean(rets[!, Symbol("r$i")]) for i in 1:n ]
R     = Matrix(rets[:, Not(:Date)])
Σ     = Symmetric(cov(R, dims=1))

println("μ = ", μ)
println("Σ = "); display(Σ)

# ─── LIQUIDITY VECTOR ─────────────────────────────────────────────────────────

println("Average weekly volumes (liquidity) = ", liquidity)


μ = [0.0006552581480219083, 0.00338988653370281, 0.0024484126990979367, 0.002592239832991949, 0.003223973087234362, 0.005013096771948148, 0.005655689162335114, 0.0014319140713245021, 0.001590768975077468, 0.010260864531881405]
Σ = 


10×10 Symmetric{Float64, Matrix{Float64}}:
  0.00186014   0.00110124   0.000760099  …   0.000999223  0.000699394
  0.00110124   0.00142203   0.000519402      0.000648364  0.000781654
  0.000760099  0.000519402  0.00159101       0.000505046  0.00061901
  0.0013958    0.00143332   0.00103061       0.000826106  0.000938006
 -0.000123485  7.45066e-5   0.000270781     -0.00010189   7.08366e-5
  0.00107309   0.00138979   0.000648584  …   0.000598146  0.000649049
  0.00099663   0.00129395   0.000644582      0.000562353  0.000767245
 -0.000118589  0.000149548  0.000206082     -0.000152874  2.27392e-5
  0.000999223  0.000648364  0.000505046      0.00124881   0.000548978
  0.000699394  0.000781654  0.00061901       0.000548978  0.00124903

Average weekly volumes (liquidity) = [1.9773766923076922e8, 1.3400782692307692e7, 2.6582938076923078e8, 4.305632115384615e7, 7.354774807692307e7, 1.126198076923077e7, 4.575230192307692e7, 3.741719423076923e7, 1.0168295384615384e8, 8.472379807692307e7]


In [22]:
# ─── SET UP & SOLVE OPTIMIZATION ──────────────────────────────────────────────

γ      = fill(100.0, n)
L      = liquidity
λ      = 0.5
w_prev = fill(1/n, n)

model = Model(Gurobi.Optimizer)
@variable(model, w[1:n])
@variable(model, z[1:n] >= 0)

@constraint(model, sum(w) == 1)
@constraint(model, w .>= 0)
@constraint(model, w .<= 1)

# |w[i] - w_prev[i]| <= z[i]
@constraint(model, [i=1:n], z[i] ≥ w[i] - w_prev[i])
@constraint(model, [i=1:n], z[i] ≥ -(w[i] - w_prev[i]))

@objective(model, Min,
    λ * w' * Σ * w
  - (1 - λ) * dot(w, μ)
  + sum( γ[i] * z[i] / L[i] for i in 1:n )
)

optimize!(model)

println("\nObjective value: ", objective_value(model))
println("Optimal portfolio weights:")
for i in 1:n
    println("  ", lpad(names[i], 10), " → ", round(value(w[i]), digits=4))
end

Set parameter Username
Set parameter LicenseID to value 2650745
Academic license - for non-commercial use only - expires 2026-04-12
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 41 rows, 20 columns and 70 nonzeros
Model fingerprint: 0xe78a283e
Model has 55 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-07, 5e-03]
  QObjective range [5e-05, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+00]
Presolve removed 20 rows and 0 columns
Presolve time: 0.00s
Presolved: 21 rows, 20 columns, 50 nonzeros
Presolved model has 55 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 9
 AA' NZ     : 1.830e+02
 Factor NZ  : 4.270e+02
 Factor Ops : 7.935e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objec